In [347]:
import pandas as pd
import numpy as np
import numpy.random as alea
import seaborn as sns

In [223]:
articles_df = pd.read_csv('./news-article-categories.csv')
articles_df.head(5)

,category,title,body
0,ARTS & CULTURE,Modeling Agencies Enabled Sexual Predators For...,"In October 2017, Carolyn Kramer received a dis..."
1,ARTS & CULTURE,Actor Jeff Hiller Talks “Bright Colors And Bol...,This week I talked with actor Jeff Hiller abou...
2,ARTS & CULTURE,New Yorker Cover Puts Trump 'In The Hole' Afte...,The New Yorker is taking on President Donald T...
3,ARTS & CULTURE,Man Surprises Girlfriend By Drawing Them In Di...,"Kellen Hickey, a 26-year-old who lives in Huds..."
4,ARTS & CULTURE,This Artist Gives Renaissance-Style Sculptures...,There’s something about combining the traditio...


In [310]:
articles_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6872 entries, 0 to 6876
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   category  6872 non-null   object
 1   title     6872 non-null   object
 2   body      6872 non-null   object
dtypes: object(3)
memory usage: 472.8+ KB


In [313]:
articles_df.head()

,category,title,body
0,ARTS & CULTURE,Modeling Agencies Enabled Sexual Predators For...,"In October 2017, Carolyn Kramer received a dis..."
1,ARTS & CULTURE,Actor Jeff Hiller Talks “Bright Colors And Bol...,This week I talked with actor Jeff Hiller abou...
2,ARTS & CULTURE,New Yorker Cover Puts Trump 'In The Hole' Afte...,The New Yorker is taking on President Donald T...
3,ARTS & CULTURE,Man Surprises Girlfriend By Drawing Them In Di...,"Kellen Hickey, a 26-year-old who lives in Huds..."
4,ARTS & CULTURE,This Artist Gives Renaissance-Style Sculptures...,There’s something about combining the traditio...


In [322]:
articles_df = articles_df.reset_index().drop(columns="index")
articles_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6872 entries, 0 to 6871
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   category  6872 non-null   object
 1   title     6872 non-null   object
 2   body      6872 non-null   object
dtypes: object(3)
memory usage: 161.2+ KB


In [225]:
articles_df = articles_df.dropna()

In [226]:
articles_df.shape[0]

6872

In [227]:
articles_df["category"].value_counts()

ARTS & CULTURE    1001
BUSINESS           501
ENTERTAINMENT      501
ENVIRONMENT        501
POLITICS           501
RELIGION           501
SPORTS             501
TECH               501
WOMEN              501
EDUCATION          490
COMEDY             376
SCIENCE            350
MEDIA              347
CRIME              300
Name: category, dtype: int64

In [228]:
from gensim import corpora, models, similarities
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk import download
download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\seni2\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [229]:
#Concatenating the articles titles and bodies
articles_content = (articles_df['title'] + ' ' + articles_df['body']).tolist()

In [242]:
#Loading a set of English stopwords
english_stopset_1 = set(stopwords.words('english')).union(
                  {"things", "that's", "there's", "something", "take", "don't", "don", "may", 
                   "set", "might", "says", "including", "lot", "much", "said", "know", "want",
                   "good", "step", "often", "going", "thing", "things", "think", "you're",
                   "back", "actually", "better", "look", "find", "right", "example", 
                   "verb", "verbs", "even", "could", "can", "get", "show", "new",
                   "get", "first", "two", "really", "way", "say", "percent", "many",
                   "see", "use", "word", "words", "us", "make", "need", "never", "love",
                   "see", "life", "go", "day", "play", "group", "former","according", "year",
                   "feel", "family", "thought", "story", "kind", "great", "felt",
                   "games", "city", "cities", "body", "night", "light", "girl", "friend",
                   "real", "person", "people", "big", "long"})

In [243]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [244]:
# Importing a larger list of stopwords
with open("./eng_stopwords.txt", "r") as f:
    eng_stopwords = f.readlines()
    #print(eng_stopwords)

for i in range(len(eng_stopwords)):
    eng_stopwords[i] = eng_stopwords[i][:-1]

english_stopset_2 = set(eng_stopwords).union(english_stopset_1)
print(english_stopset_2)

{'exactly', "we're", 'shown', 'thereupon', 'describe', 'whom', 'able', 'some', 'the', 'with', 'getting', 'specifying', 'else', 'into', 'somewhere', 'last', 'truly', 'for', 'research', "she'd", 'inc.', 'sufficiently', 'name', 'including', 'us', "i'll", 'nearly', 'noone', 'my', 'definitely', 'quite', 'between', 'here', 'containing', 'twelve', 'step', 'entirely', 'whither', 'couldn', 'what', 'little', 'towards', 'fire', 'several', 'following', 'promptly', 'especially', "there're", 'found', 'made', 'arise', 'less', 'means', 'in', 'cant', 'placed', 'howbeit', 'd', 'were', 'who', 'beginnings', 'person', 'thus', 'minus', 'hopefully', 'whats', 'hes', 'past', 'cannot', "hasn't", 'refs', "doesn't", 'not', 'directly', 'r', 'neverless', 'welcome', 'considering', 'tell', 'oh', 'now', 'keep', 'self', 'its', 'top', 'ought', 'become', 'far', 'significant', 'fewer', "who'll", 'thoroughly', 'beforehand', 'should', 'each', 'sometimes', 'say', 'shed', 'when', 'insofar', 'though', 'doesn', "mustn't", 'at',

In [245]:
#Tokenizing words of articles
tokenizer = RegexpTokenizer(r"(?u)[\b\#a-zA-Z][\w&-_]+\b")
articles_tokens = list(map(lambda d: [token for token in tokenizer.tokenize(d.lower()) if token not in english_stopset_2], articles_content))

In [246]:
#Processing bigrams from unigrams (sets of two works frequently together in the corpus)
bigram_transformer = models.Phrases(articles_tokens)
articles_unigrams_bigrams_tokens = list(bigram_transformer[articles_tokens])

In [247]:
#### Eventually use pattern librairy

#Creating a dictionary and filtering out too rare and too common tokens
english_dictionary = corpora.Dictionary(articles_unigrams_bigrams_tokens)
english_dictionary.filter_extremes(no_below=5, no_above=0.2, keep_n=None)
english_dictionary.compactify()
print(english_dictionary.most_common(100))

[('women', 5455), ('trump', 4857), ('students', 3141), ('company', 3013), ('school', 2683), ('u.s', 2435), ('children', 2076), ('president', 2041), ('country', 2002), ('art', 1980), ('public', 1870), ('american', 1853), ('support', 1821), ('place', 1808), ('education', 1781), ('change', 1747), ('schools', 1728), ('woman', 1713), ('times', 1683), ('men', 1667), ('man', 1644), ('today', 1625), ('facebook', 1605), ('case', 1584), ('york', 1581), ('wrote', 1557), ('statement', 1548), ('making', 1528), ('power', 1514), ('asked', 1506), ('report', 1498), ('america', 1459), ('book', 1422), ('news', 1422), ('working', 1409), ('team', 1406), ('fact', 1399), ('history', 1373), ('community', 1372), ('white', 1336), ('experience', 1325), ('future', 1321), ('money', 1317), ('left', 1303), ('read', 1293), ('live', 1290), ('days', 1285), ('united', 1279), ('teachers', 1275), ('political', 1273), ('business', 1271), ('national', 1270), ('point', 1263), ('video', 1254), ('program', 1251), ('data', 1251

In [248]:
#Processing Bag-of-Words (BoW) for each article
articles_bow = [english_dictionary.doc2bow(doc) for doc in articles_unigrams_bigrams_tokens]

In [250]:
#Training the LDA topic model on English articles
lda_model = models.LdaModel(articles_bow, id2word=english_dictionary, num_topics=24, passes=15, iterations=1500)

In [251]:
#Processing the topics for each article
articles_lda = lda_model[articles_bow]

In [252]:
print(articles_lda)

In [253]:
def get_topics_top_words(model, max_words):
    all_topics = model.show_topics(-1, max_words*2, False, False)
    topics = []
    for topic in all_topics:    
        min_score_word = float(abs(topic[1][0][1])) / 10.
        top_positive_words = list(map(lambda y: y[0].replace('_',' '), filter(lambda x: x[1] > min_score_word, topic[1])))[0:max_words]
        topics.append('[' + ', '.join(top_positive_words) + ']')
    return topics

#Computing the main topic of each article
topics_top_words = get_topics_top_words(lda_model, 7)

In [254]:
def get_main_topics(corpus_lda, topics_labels):
    min_strength = (1.0 / float(len(topics_labels)))/10 + 0.01
    main_topics = map(lambda ts: sorted(ts, key=lambda t: -t[1])[0][0] if sorted(ts, key=lambda t: -t[1])[0][1] > min_strength else None, corpus_lda)
    second_topics = map(lambda ts: sorted(ts, key=lambda t: -t[1])[1][0] if len(sorted(ts, key=lambda t: -t[1])) > 1 else None, corpus_lda)
    main_topics_labels = map(lambda x: topics_labels[x] if x != None else '', main_topics)
    second_topics_labels = map(lambda x: topics_labels[x] if x != None else '', second_topics)
    return list(main_topics_labels), list(second_topics_labels)

#Return the discovered topics, sorted by popularity
corpus_main_topics, corpus_second_topics = get_main_topics(articles_lda, topics_top_words)

main_topics_df = pd.DataFrame(corpus_main_topics, columns=['topic']).groupby('topic').size().sort_values(ascending=True).reset_index()
main_topics_df.columns = ['topic','count']
main_topics_df.sort_values('count', ascending=False)

,topic,count
23,"[trump, president, god, america, political, do...",856
22,"[children, friends, talk, kids, job, experienc...",785
21,"[art, artists, artist, space, human, works, pl...",773
20,"[film, season, movie, music, book, series, song]",672
19,"[students, education, school, schools, childre...",493
18,"[water, scientists, climate change, science, e...",409
17,"[apple, data, technology, google, company, app...",407
16,"[police, man, car, year-old, death, killed, bl...",366
15,"[abuse, sexual assault, statement, allegations...",355
14,"[game, team, event, players, win, sports, record]",291


In [255]:
print(len(topics_top_words))

for t in topics_top_words:
    print(t)

24
[apple, data, technology, google, company, app, iphone]
[jewish, war, white, muslims, history, jews, painting]
[case, law, court, cases, legal, usa gymnastics, judge]
[teachers, teacher, school, students, teaching, districts, gun violence]
[water, scientists, climate change, science, energy, earth, researchers]
[texas, public schools, olympics, board, jackson, bitcoin, gates]
[abuse, sexual assault, statement, allegations, survivors, wrote, women]
[post shared, samsung, abortion, abortions, animals, planned parenthood, christians]
[vote, campaign, democrats, protest, house, congress, legislation]
[police, man, car, year-old, death, killed, black]
[trump, president, god, america, political, donald trump, election]
[students, education, school, schools, children, support, community]
[facebook, social media, twitter, news, video, company, content]
[company, companies, billion, amazon, workers, business, pay]
[u.s, uber, company, fbi, russia, investigation, statement]
[game, team, event

In [256]:
cmt1 = corpus_main_topics[1]
cst1 = corpus_second_topics[1]
print(cmt1)
print(cst1)
i_cmt1 = topics_top_words.index(cmt1)
print(i_cmt1)
i_cst1 = topics_top_words.index(cst1)
print(i_cst1)
print(articles_lda[1])

[film, season, movie, music, book, series, song]
[trump, president, god, america, political, donald trump, election]
16
10
[(6, 0.02718419), (10, 0.117340475), (16, 0.8485727)]


In [266]:
n = alea.randint(articles_df.shape[0])
#n = 1
article = articles_df.iloc[n]
print(article['category'])
print(article['title'])
print(article['body'][:500])
print()

main_topics = corpus_main_topics[n]
second_topics = corpus_second_topics[n]
print("topic group " + str(topics_top_words.index(corpus_main_topics[n])))
print(main_topics)
print("topic group " + str(topics_top_words.index(corpus_second_topics[n])))
print(second_topics)
print("scores :")
print(sorted(articles_lda[n], key=lambda t: -t[1]))

ARTS & CULTURE
Midcentury in Paris: A Visual Map of Modern Artists
From the winding streets of Montmarte to cabaret-canvassed Pigalle, many of the world's most renowned talents inhabited the French capital. Here, an essential who's-who of the early Paris art scene.

 Much like the 1970s punk rock scene belonged to New York and Los Angeles was the birthplace of 1950s film noir, no locale is more synonymous with Modernism than Paris. Aspiring and established artists alike flocked from around the world to access the creative energy, visual innovation and flouris

topic group 20
[art, artists, artist, space, human, works, place]
topic group 1
[jewish, war, white, muslims, history, jews, painting]
scores :
[(20, 0.48487017), (1, 0.19475868), (11, 0.09529906), (0, 0.06722766), (12, 0.059935607), (15, 0.05215109), (4, 0.029675616), (5, 0.010967277)]


In [267]:
categories = articles_df["category"].unique()
for i in range(len(categories)):
    print(str(i) + ' ' + categories[i])

0 ARTS & CULTURE
1 BUSINESS
2 COMEDY
3 CRIME
4 EDUCATION
5 ENTERTAINMENT
6 ENVIRONMENT
7 MEDIA
8 POLITICS
9 RELIGION
10 SCIENCE
11 SPORTS
12 TECH
13 WOMEN


In [269]:
CategoryLinks = [12, 9, 3, 4, 6, 5, 13, 2, 8, 3, 8, 4, 7, 1, 8, 11, 0, 5, 2, 8, 0, 9, 3, 13]

In [293]:
for i in range(len(topics_top_words)):
    print(str(i) + ' ' + topics_top_words[i])

0 [apple, data, technology, google, company, app, iphone]
1 [jewish, war, white, muslims, history, jews, painting]
2 [case, law, court, cases, legal, usa gymnastics, judge]
3 [teachers, teacher, school, students, teaching, districts, gun violence]
4 [water, scientists, climate change, science, energy, earth, researchers]
5 [texas, public schools, olympics, board, jackson, bitcoin, gates]
6 [abuse, sexual assault, statement, allegations, survivors, wrote, women]
7 [post shared, samsung, abortion, abortions, animals, planned parenthood, christians]
8 [vote, campaign, democrats, protest, house, congress, legislation]
9 [police, man, car, year-old, death, killed, black]
10 [trump, president, god, america, political, donald trump, election]
11 [students, education, school, schools, children, support, community]
12 [facebook, social media, twitter, news, video, company, content]
13 [company, companies, billion, amazon, workers, business, pay]
14 [u.s, uber, company, fbi, russia, investigatio

In [330]:
corpus_lda_category_index = [topics_top_words.index(topic) for topic in corpus_main_topics]
corpus_lda_category_topics = [str(topic) for topic in corpus_main_topics]

corpus_converted_category = [categories[CategoryLinks[i]] for i in corpus_lda_category_index]
print(len(corpus_converted_category))
print(corpus_converted_category[:100])

6872
['WOMEN', 'ARTS & CULTURE', 'WOMEN', 'ARTS & CULTURE', 'ARTS & CULTURE', 'ARTS & CULTURE', 'WOMEN', 'SPORTS', 'ARTS & CULTURE', 'ARTS & CULTURE', 'ARTS & CULTURE', 'ARTS & CULTURE', 'POLITICS', 'RELIGION', 'ARTS & CULTURE', 'ARTS & CULTURE', 'ARTS & CULTURE', 'ARTS & CULTURE', 'BUSINESS', 'WOMEN', 'POLITICS', 'RELIGION', 'ARTS & CULTURE', 'ARTS & CULTURE', 'ENVIRONMENT', 'ARTS & CULTURE', 'ENVIRONMENT', 'ARTS & CULTURE', 'WOMEN', 'ARTS & CULTURE', 'ARTS & CULTURE', 'ARTS & CULTURE', 'WOMEN', 'ARTS & CULTURE', 'CRIME', 'SPORTS', 'ARTS & CULTURE', 'RELIGION', 'WOMEN', 'ARTS & CULTURE', 'POLITICS', 'WOMEN', 'ARTS & CULTURE', 'ARTS & CULTURE', 'SPORTS', 'EDUCATION', 'ENTERTAINMENT', 'RELIGION', 'ARTS & CULTURE', 'RELIGION', 'ARTS & CULTURE', 'ARTS & CULTURE', 'ARTS & CULTURE', 'ARTS & CULTURE', 'RELIGION', 'ARTS & CULTURE', 'COMEDY', 'EDUCATION', 'POLITICS', 'ARTS & CULTURE', 'ARTS & CULTURE', 'ENTERTAINMENT', 'ARTS & CULTURE', 'ARTS & CULTURE', 'ARTS & CULTURE', 'COMEDY', 'EDUCATION'

In [331]:
converted_category_df = pd.DataFrame(corpus_converted_category, columns=["computed_category"])
lda_category_index_df = pd.DataFrame(corpus_lda_category_index, columns=["lda_category_index"])
lda_category_topics_df = pd.DataFrame(corpus_lda_category_topics , columns=["lda_category_topics"])
new_category_df = pd.concat([converted_category_df, lda_category_index_df, lda_category_topics_df], axis=1)
print(new_category_df.shape)
new_category_df.info()

(6872, 3)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6872 entries, 0 to 6871
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   computed_category    6872 non-null   object
 1   lda_category_index   6872 non-null   int64 
 2   lda_category_topics  6872 non-null   object
dtypes: int64(1), object(2)
memory usage: 161.2+ KB


In [346]:
category_df = pd.concat([articles_df,new_category_df],axis=1)
category_df.head()

,category,title,body,computed_category,lda_category_index,lda_category_topics
0,ARTS & CULTURE,Modeling Agencies Enabled Sexual Predators For...,"In October 2017, Carolyn Kramer received a dis...",WOMEN,6,"[abuse, sexual assault, statement, allegations..."
1,ARTS & CULTURE,Actor Jeff Hiller Talks “Bright Colors And Bol...,This week I talked with actor Jeff Hiller abou...,ARTS & CULTURE,16,"[film, season, movie, music, book, series, song]"
2,ARTS & CULTURE,New Yorker Cover Puts Trump 'In The Hole' Afte...,The New Yorker is taking on President Donald T...,WOMEN,6,"[abuse, sexual assault, statement, allegations..."
3,ARTS & CULTURE,Man Surprises Girlfriend By Drawing Them In Di...,"Kellen Hickey, a 26-year-old who lives in Huds...",ARTS & CULTURE,20,"[art, artists, artist, space, human, works, pl..."
4,ARTS & CULTURE,This Artist Gives Renaissance-Style Sculptures...,There’s something about combining the traditio...,ARTS & CULTURE,20,"[art, artists, artist, space, human, works, pl..."


In [333]:
category_df[category_df["category"]==category_df["computed_category"]].shape[0]/(category_df.shape[0])

0.37849243306169966

In [345]:
reduced_category_df = category_df[["category", "lda_category_index", "lda_category_topics"]]
count_category_df = reduced_category_df.groupby(["category", "lda_category_index"], sort=True).count()
count_category_df[count_category_df]

lda_category_topics
category       lda_category_index                     
ARTS & CULTURE 0                                     6
               1                                    39
               2                                     2
               4                                     7
               5                                     1
...                                                ...
WOMEN          17                                  110
               18                                    9
               20                                   11
               22                                    1
               23                                  106

[280 rows x 1 columns]

In [ ]:
ax = sns.heatmap()